this notebook sets up the data cleaning after submissions are pulled. Lines that are commented out were first tried here, and decided to be better placed in the intial function within the requests notebook

In [1]:
import pandas as pd

In [2]:
dem = pd.read_csv('./data/democrat100k.csv')
rep = pd.read_csv('./data/conservative100k.csv')

/home/dale/anaconda3/envs/python397/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3,5,7,10,12,13,21,23,26,29,46,62,65,76,78,79,81,84,85,86,88,89,90,92,93,94,95,96) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/dale/anaconda3/envs/python397/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = pd.concat([dem,rep])

In [4]:

df['removed_by_category'].value_counts()

automod_filtered    31594
moderator           22073
reddit              10000
deleted              2057
author                  4
Name: removed_by_category, dtype: int64

Removing posts that were filtered by automod (very likely bot posts), moderator (not following subreddit rules or potential bot), reddit (likely more serious), and author + deleted as the content of the post may have been poor or the author might not be active and would not be as valuable to predictions <br>
saving ID so that if comments are pulled later, comments for such posts can be filtered earlier on <br>

will likely be integrated into loop later on, so this doesnt have to be done for each pull, and since over half the posts are filtered, many more pulls will likely be required.

In [5]:
filter_cat = ['automod_filtered', 'moderator', 'reddit','author','deleted']


filtered_authors =  df[df['removed_by_category'].isin(filter_cat)]['author'].unique()

df =  df[~df['removed_by_category'].isin(filter_cat)]

In [6]:
filtered_authors = list(filtered_authors)

check to see if filtered authors still have active posts on reddit, so older submissions don't need to filtered by this list

In [7]:
df['author'].isin(filtered_authors).sum()


49247

Most of the filtered posts are from authors who still have active posts in subreddit<br>
Check to see if DF has no deleted posts

In [8]:
df['removed_by_category'].value_counts()

Series([], Name: removed_by_category, dtype: int64)

In [9]:
df = df[['selftext','created_utc','title','id','author','subreddit']]

In [10]:
df.isnull().sum()

selftext       67042
created_utc        0
title              0
id                 0
author             0
subreddit          0
dtype: int64

In [11]:
len(df)

72959

In [12]:
title_self = df[df['selftext'].notnull()]

In [13]:
title_self

,selftext,created_utc,title,id,author,subreddit
971,"Greetings, r/democrats. Recently we've receive...",1639631272,Announcement: Self/text posts are now disabled,rhj7bx,NickHancock,democrats
973,You’ve all heard them. But what’s gonna happen...,1639627277,How do anti-vax people expect to live normally...,rhhyin,DumplingsandDynamics,democrats
976,Democrats should be pushing the Build Back Bet...,1639615153,"Build Back Better, Kentucky!!!",rhe13t,notevergreens,democrats
987,He can forgive student loans. While he may ne...,1639597351,I am a parent scraping by who is about to take...,rh7hkg,hoyfkd,democrats
1009,[deleted],1639559096,Testing,rguxim,[deleted],democrats
...,...,...,...,...,...,...
67527,Mainstream media crushes Biden for ‘flat-foote...,1629131537,You know it is bad when he has lost the drive ...,p5jnb1,SouthernBoat2109,Conservative
67531,The disinformation campaign to excuse Biden of...,1629131361,Mods need to make every post flair-only today,p5jl5x,StuffyKnows2Much,Conservative
68775,$oros and Hates acquiring vax testing tells y...,1628878401,The crickets chirp,p3qsr4,fishntools,Conservative
69172,My theory is that the governments of countries...,1628796064,Is Universal Basic Income tied to Government-f...,p36t2b,br54987654321,Conservative


Going to paste title and selftext since not all have selftext but could be useful context, remove [deleted] first

In [14]:
df = df[df['selftext'] != '[deleted]']

also check to see if title has any [deleted]


In [15]:
df[df['title'] == '[deleted]'].sum().sum()

TypeError: can only concatenate str (not "numpy.int64") to str

In [16]:
df['title_self'] = df['title'] + df['selftext']

In [17]:
df['sub_bin'] = df['subreddit'].map({'democrats':0, 'Conservative': 1})

In [18]:
df['sub_bin'].value_counts()

0    37258
1    35662
Name: sub_bin, dtype: int64

In [19]:
df.to_csv('./data/posts_cleaned_100k.csv')